In [37]:
import torch
from torchvision import models
import torch.nn as nn
import json
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from torchvision import transforms
import torch.optim as optim

In [39]:
def load_labels(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)  # JSON 파일을 로드합니다.
    return data

In [41]:
class FaceDataset(Dataset):
    def __init__(self, json_file, root_dir, transform=None):
        self.data = load_labels(json_file)  # JSON 파일에서 데이터 로드
        self.root_dir = root_dir  # 데이터의 기본 경로
        self.transform = transform  # 이미지 전처리

        # 이미지 경로와 레이블을 리스트로 저장
        self.image_paths = list(self.data.keys())  # 이미지 경로 리스트
        self.labels = list(self.data.values())  # 레이블 리스트

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_paths[idx])  # 이미지 경로
        if not os.path.exists(img_path):  # 경로가 존재하는지 체크
            print(f"Warning: Image {img_path} not found.")
            return None, None  # 파일이 없으면 None 반환

        image = Image.open(img_path).convert("RGB")  # 이미지를 열고 RGB로 변환
        label = self.labels[idx]  # 레이블

        if self.transform:
            image = self.transform(image)  # 변환 적용

        # CrossEntropyLoss가 요구하는 형태로 레이블을 1D 텐서로 변환
        label = torch.tensor(label, dtype=torch.long)  # 정수형 텐서로 변환

        return image, label

        

In [54]:
# 데이터셋 경로 설정
train_json_path = "C:\\Users\\user\\Desktop\\deepface-master\\metas\\protocol2\\test_on_high_quality_device\\train_label.json"
test_json_path = "C:\\Users\\user\\Desktop\\deepface-master\\metas\\protocol2\\test_on_high_quality_device\\test_label.json"

# 훈련 데이터셋 로딩
train_data = FaceDataset(json_file=train_json_path, root_dir="C:\\Users\\user\\Desktop\\deepface-master", transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

# 테스트 데이터셋 로딩
test_data = FaceDataset(json_file=test_json_path, root_dir="C:\\Users\\user\\Desktop\\deepface-master", transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\deepface-master\\metas\\protocol2\\test_on_high_quality_device\\train_label.json'

In [45]:
# 데이터로더 설정
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

NameError: name 'train_data' is not defined

In [47]:
# 사전 학습된 모델 불러오기
model = models.resnet50(pretrained=True)

# 출력층을 이진 분류에 맞게 변경 (2개의 클래스: live, spoof)
model.fc = nn.Linear(model.fc.in_features, 2)

C:\Users\user\.conda\envs\deepface\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\.conda\envs\deepface\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
# 모델을 GPU로 이동 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [21]:
# 손실 함수 (크로스 엔트로피 손실)
criterion = nn.CrossEntropyLoss()

# 옵티마이저 (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            if inputs is None or labels is None:
                continue  # None 값이 있으면 건너뜁니다.

            # 모델 입력, 레이블을 GPU로 이동
            inputs, labels = inputs.to(device), labels.to(device)

            # 기울기 초기화
            optimizer.zero_grad()

            # 순전파
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 역전파
            loss.backward()
            optimizer.step()

            # 결과 출력
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # 에폭 종료 후 결과 출력
        accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

In [23]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            if inputs is None or labels is None:
                continue  # None 값이 있으면 건너뜁니다.
            
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [24]:
# 모델 저장
torch.save(model.state_dict(), 'face_spoof_model.pth')
print("Model saved to 'face_spoof_model.pth'")

# 모델 로드 (CPU에서 로드하도록 명시)
model.load_state_dict(torch.load('face_spoof_model.pth', map_location=device))
model.eval()  # 모델을 평가 모드로 변경

Model saved to 'face_spoof_model.pth'


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [58]:
# 모델 정의
class FaceSpoofModel(nn.Module):
    def __init__(self):
        super(FaceSpoofModel, self).__init__()
        # 모델 계층 정의
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 224 * 224, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.maxpool(self.relu(self.conv1(x)))
        x = self.maxpool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 객체 생성
model = FaceSpoofModel()

# 모델 구조 출력
print(model)

FaceSpoofModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1605632, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [59]:
import torch
from torch import nn
from torchvision import transforms
from PIL import Image

# 모델 정의 (주어진 구조에 맞게 수정)
class FaceSpoofModel(nn.Module):
    def __init__(self):
        super(FaceSpoofModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(2, 2)  # MaxPooling 적용
        self.fc1 = nn.Linear(32 * 56 * 56, 512)  # fc1의 입력 크기 수정
        self.fc2 = nn.Linear(512, 2)  # 최종 출력 2개 (진짜, 가짜)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)  # 첫 번째 MaxPool
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)  # 두 번째 MaxPool
        x = x.view(-1, 32 * 56 * 56)  # 평탄화
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        
# ResNet-50 모델 로드 (pretrained=False로 초기화)
model = models.resnet50(pretrained=False)

# fc 레이어를 이진 분류에 맞게 수정
model.fc = torch.nn.Linear(model.fc.in_features, 2)  # 2는 클래스의 개수 (이진 분류)

# 저장된 모델 파라미터 불러오기
model.load_state_dict(torch.load('face_spoof_model.pth'))

# 평가 모드로 설정
model.eval()

# 이제 모델을 사용하여 예측 등을 진행할 수 있습니다.
# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 텐서 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

# 예측 함수
def predict_image(image_path):
    # 이미지 열기
    img = Image.open(image_path)
    
    # 이미지 전처리
    img_tensor = transform(img).unsqueeze(0)  # 배치 차원 추가
    
    # 예측
    with torch.no_grad():  # 학습을 하지 않도록 설정
        output = model(img_tensor)
    
    # 출력값이 0일 경우 '진짜', 1일 경우 '가짜'로 분류
    _, predicted = torch.max(output, 1)
    if predicted.item() == 0:
        return "진짜"
    else:
        return "가짜"

# 이미지 경로 설정
image_path = "C:\\Users\\user\\Desktop\\test.jpg"  # 실제 테스트할 이미지 경로로 변경

# 예측 실행
result = predict_image(image_path)
print(f'이 이미지는 "{result}"입니다.')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\Desktop\\test.jpg'

In [2]:
import torch
import torch.nn as nn  # nn 모듈 임포트 추가
import cv2
import numpy as np
import os
from torchvision import transforms
from PIL import Image
from deepface import DeepFace

# FaceSpoofModel 클래스 정의
class FaceSpoofModel(nn.Module):
    def __init__(self):
        super(FaceSpoofModel, self).__init__()
        # 합성곱 층 정의
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(2, 2)  # 최대 풀링 층
        self.fc1 = nn.Linear(32 * 56 * 56, 512)  # 완전 연결층
        self.fc2 = nn.Linear(512, 2)  # 출력층
        self.relu = nn.ReLU()  # ReLU 활성화 함수

    def forward(self, x):
        # 순전파 과정
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.view(-1, 32 * 56 * 56)  # 텐서 형태 변경
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = FaceSpoofModel()
model.load_state_dict(torch.load('face_spoof_model.pth'))  # 모델 가중치 로드
model.eval()  # 평가 모드로 설정

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

def highlight_facial_areas(img: np.ndarray, faces_coordinates: list[tuple[int, int, int, int]], anti_spoofing: bool = True) -> np.ndarray:
    """ 얼굴 영역에 박스를 그리는 함수 """
    for x, y, w, h in faces_coordinates:
        color = (0, 255, 0) if anti_spoofing else (0, 0, 255)  # 진짜/가짜에 따라 색상 변경
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)  # 사각형 그리기
    return img

def is_real_face(emotion_probas: dict) -> bool:
    """ 표정이 'happy' 또는 'neutral'일 경우 실제 얼굴로 판단 """
    dominant_emotion = max(emotion_probas, key=emotion_probas.get)  # 가장 높은 확률의 감정 찾기
    return dominant_emotion in ["happy", "neutral"]  # 해당 감정인지 확인

def predict_image(img: np.ndarray) -> str:
    """ 이미지에서 진짜/가짜 얼굴 예측 """
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # OpenCV 이미지를 PIL 이미지로 변환
    img_tensor = transform(img_pil).unsqueeze(0)  # 전처리 후 텐서 변환

    with torch.no_grad():  # 기울기 계산 비활성화
        output = model(img_tensor)  # 모델 예측

    _, predicted = torch.max(output, 1)  # 예측된 클래스 찾기
    return "진짜" if predicted.item() == 0 else "가짜"  # 클래스에 따라 결과 반환

# 바탕화면 경로 설정
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
photo_count = 0  # 찍은 사진 수

# 카메라 실행
cap = cv2.VideoCapture(0)
reference_img_path = "tests/dataset/img14.jpg"  # 비교할 등록된 얼굴 이미지

while cap.isOpened():
    ret, frame = cap.read()  # 프레임 읽기
    if not ret:
        break

    try:
        # 얼굴 감지 및 분석
        demographies = DeepFace.analyze(frame, actions=("age", "gender", "emotion"), enforce_detection=False)

        if demographies:
            demography = demographies[0]  # 첫 번째 분석 결과 가져오기
            x, y, w, h = demography["region"]["x"], demography["region"]["y"], demography["region"]["w"], demography["region"]["h"]

            # 표정 기반 안티 스푸핑 및 본인 여부 판별
            is_real = is_real_face(demography["emotion"])  # 실제 얼굴인지 판별
            face_region = frame[y:y+h, x:x+w]  # 얼굴 영역 추출
            face_prediction = predict_image(face_region)  # 얼굴 예측
            is_matched = (face_prediction == "진짜") and is_real  # 진짜 얼굴 여부 확인

            if is_matched:
                frame = highlight_facial_areas(frame, [(x, y, w, h)], anti_spoofing=True)  # 진짜 얼굴 박스 그리기
                status_text = "Real Face"  # 상태 텍스트
            else:
                frame = highlight_facial_areas(frame, [(x, y, w, h)], anti_spoofing=False)  # 가짜 얼굴 박스 그리기
                status_text = "Fake Face"  # 상태 텍스트

            # 감정 상태 텍스트 추가
            cv2.putText(frame, status_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # 화면 출력
        cv2.imshow("Face Recognition", frame)

        # 'c' 키를 누르면 사진 찍기
        if cv2.waitKey(1) & 0xFF == ord('c'):
            photo_count += 1
            photo_path = os.path.join(desktop_path, f"photo_{photo_count}.jpg")  # 사진 저장 경로
            cv2.imwrite(photo_path, frame)  # 사진 저장
            print(f"사진 {photo_count}장이 찍혔습니다. 저장 경로: {photo_path}")
        
        # 5장을 찍으면 종료
        if photo_count >= 5:
            print("5장 사진이 찍혔습니다. 프로그램을 종료합니다.")
            break

    except Exception as e:
        print(f"Error: {e}")  # 에러 출력

# 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  # 카메라 해제
cv2.destroyAllWindows()  # 모든 윈도우 닫기


RuntimeError: Error(s) in loading state_dict for FaceSpoofModel:
	Missing key(s) in state_dict: "conv1.bias", "conv2.weight", "conv2.bias", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 
	Unexpected key(s) in state_dict: "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn1.num_batches_tracked", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.bn1.num_batches_tracked", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.bn2.num_batches_tracked", "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.bn3.num_batches_tracked", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.0.downsample.1.num